In [ ]:
#Notebook is running in tmux
import socket
hostname = socket.gethostname()
print(hostname)

In [ ]:
import ROOT
from ROOT import TMVA
from array import array

In [ ]:
#inputFileS = ROOT.TFile("sig_1000.root")
dirNtuple = "root://cmseos.fnal.gov//store/user/rverma/Output/cms-TT-run2/Ntuple_Skim/"
dirFile = "2016/Semilep/JetBase/"
sigFile = "Semilep_JetBase__TstarTstarToTgammaTgluon_M800_2016_Ntuple.root"
inputFileS = ROOT.TFile.Open("%s/%s/%s"%(dirNtuple, dirFile, sigFile))
sig = inputFileS.Get("AnalysisTree")

bkg = ROOT.TChain("AnalysisTree")
bkgList = ["TTGamma_SingleLept", 
           "TTGamma_Dilepton", 
           "TTGamma_Hadronic", 
           "TTGamma_SingleLept_Pt100", 
           "TTGamma_Dilepton_Pt100", 
           "TTGamma_Hadronic_Pt100",
           "TTGamma_SingleLept_Pt200", 
           "TTGamma_Dilepton_Pt200",
           "TTGamma_Hadronic_Pt200"
          ]
for b in bkgList:
    fPath = "%s/%s/Semilep_JetBase__%s_2016_Ntuple.root"%(dirNtuple, dirFile, b)
    bkg.Add(fPath)
print(bkg.GetEntries())

In [ ]:
Reco_ht = array('f',[0])
Reco_st = array('f',[0])
Photon_et = array('f',[0])
Reco_mass_T = array('f',[0])
Weight_lumi = array('f',[0])

methodName = "BDTG_mT800"
weightfile = "dataset/weights/TMVA_TT_Classification_" + methodName + ".weights.xml"

ROOT.TMVA.Tools.Instance()
reader = TMVA.Reader("!Color:!Silent")
reader.AddVariable("Reco_ht", Reco_ht)
reader.AddVariable("Reco_st", Reco_st)
reader.AddVariable("Photon_et", Photon_et);
reader.AddVariable("Reco_mass_T",Reco_mass_T)
reader.BookMVA( methodName,weightfile)

In [ ]:
hSig_disc = ROOT.TH1D("BDT_Disc","BDT_Disc",40,-1,1)
hSig_Reco_ht = ROOT.TH1D("Reco_ht","Reco_ht",300,0,9000)
hSig_Photon_et = ROOT.TH1D("Photon_et","Photon_et",100,0,2000)
hSig_Reco_st = ROOT.TH1D("Reco_st","Reco_st",300,0,9000)
hSig_Reco_mass_T = ROOT.TH1D("Reco_mass_T","Reco_mass_T",300,0,6000)

hSig_Reco_mass_T_cut = ROOT.TH1D("Reco_mass_T_cut","Reco_mass_T_cut",300,0,6000)
hSig_Reco_st_cut = ROOT.TH1D("Reco_st_cut","Reco_st_cut",300,0,9000)

hBkg_disc = ROOT.TH1D("BDT_Disc","BDT_Disc",40,-1,1)
hBkg_Reco_ht = ROOT.TH1D("Reco_ht","Reco_ht",300,0,9000)
hBkg_Photon_et = ROOT.TH1D("Photon_et","Photon_et",100,0,2000)
hBkg_Reco_st = ROOT.TH1D("Reco_st","Reco_st",300,0,9000)
hBkg_Reco_mass_T = ROOT.TH1D("Reco_mass_T","Reco_mass_T",300,0,6000)

hBkg_Reco_mass_T_cut = ROOT.TH1D("Reco_mass_T_cut","Reco_mass_T_cut",300,0,6000)
hBkg_Reco_st_cut = ROOT.TH1D("Reco_st_cut","Reco_st_cut",300,0,9000)

ievt = 0
ent = sig.GetEntries()
for e in sig:
    #eventSel = e.Event_pass_presel_mu and e.Event_pass_presel_mu
    eventSel = int(e.Event_pass_presel_mu and ((e.Jet_size>=5 and e.FatJet_size==0) or (e.Jet_size>=2 and e.FatJet_size==1))  and e.Jet_b_size >=1 and e.Photon_size==1 and e.Photon_et[0] > 100)
    if eventSel>0:
        Reco_mass_T[0] = e.Reco_mass_T
        Reco_st[0] = e.Reco_st
        Reco_ht[0] = e.Reco_ht
        Photon_et[0] = e.Photon_et[0]
        output = reader.EvaluateMVA(methodName)
        if (ievt%100)==0:
            print('Event = %i/%i, Reco_mass_T = %i, MVA output = %s'%(ievt, ent, Reco_mass_T[0], output))
        ievt += 1
        
        hSig_disc.Fill(output, e.Weight_lumi)
        hSig_Reco_ht.Fill(e.Reco_ht, e.Weight_lumi)
        hSig_Reco_st.Fill(e.Reco_st, e.Weight_lumi)
        hSig_Reco_mass_T.Fill(e.Reco_mass_T, e.Weight_lumi)
        hSig_Photon_et.Fill(e.Photon_et[0], e.Weight_lumi)
        if output>0:
            hSig_Reco_mass_T_cut.Fill(e.Reco_mass_T, e.Weight_lumi)
            hSig_Reco_st_cut.Fill(e.Reco_st, e.Weight_lumi)
        
print("\nRunning for Bkg...\n")    
ievt = 0
ent = bkg.GetEntries()
for e in bkg:
    #eventSel = e.Event_pass_presel_mu and e.Event_pass_presel_mu
    eventSel = int(e.Event_pass_presel_mu and ((e.Jet_size>=5 and e.FatJet_size==0) or (e.Jet_size>=2 and e.FatJet_size==1))  and e.Jet_b_size >=1 and e.Photon_size==1 and e.Photon_et[0] > 100)
    if eventSel>0:
        Reco_mass_T[0] = e.Reco_mass_T
        Reco_st[0] = e.Reco_st
        Reco_ht[0] = e.Reco_ht
        Photon_et[0] = e.Photon_et[0]
        output = reader.EvaluateMVA(methodName)
        if (ievt%1000)==0:
            print('Event = %i/%i, Reco_mass_T = %i, MVA output = %s'%(ievt, ent, Reco_mass_T[0], output))
        ievt += 1
        hBkg_disc.Fill(output, e.Weight_lumi)
        hBkg_Reco_ht.Fill(e.Reco_ht, e.Weight_lumi)
        hBkg_Reco_st.Fill(e.Reco_st, e.Weight_lumi)
        hBkg_Reco_mass_T.Fill(e.Reco_mass_T, e.Weight_lumi)
        hBkg_Photon_et.Fill(e.Photon_et[0], e.Weight_lumi)
        if output>0:
            hBkg_Reco_mass_T_cut.Fill(e.Reco_mass_T, e.Weight_lumi)
            hBkg_Reco_st_cut.Fill(e.Reco_st, e.Weight_lumi)

        

In [ ]:
%jsroot on

c = ROOT.TCanvas()
c.Divide(2, 2)
ROOT.gPad.Draw()
c.cd(1)
ROOT.gPad.SetLogy(True)
ROOT.gStyle.SetOptStat(0)
#hSig_disc.Scale(20)
hBkg_Reco_ht.SetLineColor(2)
hSig_Reco_ht.SetLineWidth(2)
hSig_Reco_ht.GetYaxis().SetRangeUser(0, 100)
hSig_Reco_ht.Draw()
hBkg_Reco_ht.SetLineWidth(2)
hBkg_Reco_ht.Draw("SAME")

c.cd(2)
ROOT.gPad.SetLogy(True)
#hSig_Reco_st.Scale(20)
hBkg_Reco_st.SetLineColor(2)
hSig_Reco_st.SetLineWidth(2)
hSig_Reco_st.GetYaxis().SetRangeUser(0, 100)
hSig_Reco_st.Draw()
hBkg_Reco_st.SetLineWidth(2)
hBkg_Reco_st.Draw("SAME")

c.cd(3)
ROOT.gPad.SetLogy(True)
#hSig_Photon_et.Scale(20)
hBkg_Photon_et.SetLineColor(2)
hSig_Photon_et.SetLineWidth(2)
hSig_Photon_et.GetYaxis().SetRangeUser(0, 100)
hSig_Photon_et.Draw()
hBkg_Photon_et.SetLineWidth(2)
hBkg_Photon_et.Draw("SAME")

c.cd(4)
ROOT.gPad.SetLogy(True)
#hSig_Reco_mass_T.Scale(20)
hBkg_Reco_mass_T.SetLineColor(2)
hSig_Reco_mass_T.SetLineWidth(2)
hSig_Reco_mass_T.GetYaxis().SetRangeUser(0, 100)
hSig_Reco_mass_T.Draw()
hBkg_Reco_mass_T.SetLineWidth(2)
hBkg_Reco_mass_T.Draw("SAME")

In [ ]:
%jsroot on
c2 = ROOT.TCanvas()
c2.cd()
ROOT.gPad.SetLogy(True)
ROOT.gPad.Draw()
#hSig_disc.Scale(20)
hBkg_disc.SetLineColor(2)
hSig_disc.SetLineWidth(2)
hSig_disc.GetYaxis().SetRangeUser(0, 100)
hSig_disc.Draw()
hBkg_disc.SetLineWidth(2)
hBkg_disc.Draw("SAME")

In [ ]:
%jsroot on
c3 = ROOT.TCanvas()
c3.cd()
ROOT.gPad.SetLogy(True)
ROOT.gPad.Draw()
#hSig_disc.Scale(20)
hBkg_Reco_mass_T_cut.SetLineColor(2)
hSig_Reco_mass_T_cut.SetLineWidth(2)
hSig_Reco_mass_T_cut.GetYaxis().SetRangeUser(0, 100)
hSig_Reco_mass_T_cut.Draw()
hBkg_Reco_mass_T_cut.SetLineWidth(2)
hBkg_Reco_mass_T_cut.Draw("SAME")

In [ ]:
%jsroot on
c3 = ROOT.TCanvas()
c3.cd()
ROOT.gPad.SetLogy(True)
ROOT.gPad.Draw()
#hSig_disc.Scale(20)
hBkg_Reco_st_cut.SetLineColor(2)
hSig_Reco_st_cut.SetLineWidth(2)
hSig_Reco_st_cut.GetYaxis().SetRangeUser(0, 100)
hSig_Reco_st_cut.Draw()
hBkg_Reco_st_cut.SetLineWidth(2)
hBkg_Reco_st_cut.Draw("SAME")

In [ ]:
outputFile = ROOT.TFile("Disc_Ntuple.root","RECREATE")
CR = "ttyg_Enriched_SR"
def getHistDir(sample, sysType, CR):
    histDir = "%s/%s/%s"%(sample, CR, sysType)
    return histDir

def writeHist(hist, procDir, outputFile):
    outHistDir = getHistDir(procDir, "Base", CR)
    if not outputFile.GetDirectory(outHistDir):
        outputFile.mkdir(outHistDir)
    outputFile.cd(outHistDir)
    ROOT.gDirectory.Delete("%s;*"%(hist.GetName()))
    print "%20s, %10s, %10s"%(hist.GetName(), procDir, round(hist.Integral()))
    #hNew = hist.Rebin(len(newBins)-1, histNewName, newBins) 
    #hNew.Write()
    hist.Write()


procList = ["Signal", "Background"]
histList = ["Reco_st", "Reco_ht", "Photon_et", "Reco_mass_T"]

writeList = []
writeList.append([hSig_disc, "Sig", "Base"])
writeList.append([hSig_Reco_st, "Sig", "Base"])
writeList.append([hSig_Reco_ht, "Sig", "Base"])
writeList.append([hSig_Photon_et, "Sig", "Base"])
writeList.append([hSig_Reco_mass_T, "Sig", "Base"])
writeList.append([hSig_Reco_mass_T_cut, "Sig", "Base"])
writeList.append([hSig_Reco_st_cut, "Sig", "Base"])

writeList.append([hBkg_disc, "Bkg", "Base"])
writeList.append([hBkg_Reco_st, "Bkg", "Base"])
writeList.append([hBkg_Reco_ht, "Bkg", "Base"])
writeList.append([hBkg_Photon_et, "Bkg", "Base"])
writeList.append([hBkg_Reco_mass_T, "Bkg", "Base"])
writeList.append([hBkg_Reco_mass_T_cut, "Bkg", "Base"])
writeList.append([hBkg_Reco_st_cut, "Bkg", "Base"])

for write in writeList:
    writeHist(write[0], write[1], outputFile)
    if "Bkg" in write[1]:
        writeHist(write[0], "data_obs", outputFile)

outputFile.ls()
outputFile.Close()